# 1. Querying SPARQL-EndPoint

In [1]:
import pandas as pd
import numpy as np
from SPARQLWrapper import SPARQLWrapper, JSON

In [3]:
#Opening a TSV file and storing it on a numpy array
#Starting with YAGO valid file
filename = '/media/rivas/Data1/Data-mining/Kcap/datasets/yago10/test.txt' #'./yago10/train.txt'
df = np.loadtxt(open(filename,'r', encoding='utf-8'), dtype="str", delimiter="\t")
print(len(df))
print(df.shape)
print(df[0])

UnicodeEncodeError: 'latin-1' codec can't encode character '\u0107' in position 14: ordinal not in range(256)

In [ ]:
#Creating a dicctionary of subjects and predicates
sod = dict()
for x in range(0, len(df)):
    s = df[x][0]
    p = df[x][1]
    o = df[x][2]
    if (s in sod):
        sod[s] = sod[s] + 1
    else:
        sod[s] = 1
    if (p in sod):
        sod[p] = sod[p] + 1
    else:
        sod[p] = 1
    if (o in sod):
        sod[o] = sod[o] + 1
    else:
        sod[o] = 1

print(len(sod))

In [ ]:
# Iterating over keys 
sparql = SPARQLWrapper("https://linkeddata1.calcul.u-psud.fr/sparql")

count = 0
f = open("./yago10/types_test.tsv","w+", encoding='utf-8')

for state in sod: 
    print(state)
    sparql.setQuery("""
        PREFIX yago: <http://yago-knowledge.org/resource/>
        PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
        SELECT DISTINCT ?obj
        WHERE { 
            <http://yago-knowledge.org/resource/%s> rdf:type ?obj
        }
        """ % state)
    sparql.setReturnFormat(JSON)
    results = sparql.query().convert()

    for result in results["results"]["bindings"]:
        f.write("%s\t%s\t%s\n" % (state, "rdf:type", result["obj"]["value"].replace("http://yago-knowledge.org/resource/","")))
        count = count + 1
        #print('%s' % (result["obj"]["value"]))

f.close()
print(count)        

# 2. Transforming training data to RDF

In [39]:
#Opening a TSV file and storing it on a numpy array
#Starting with YAGO valid file
filename = './yago10/types/types_test_004.tsv' #'./yago10/train.txt'
df = np.loadtxt(open(filename,'r', encoding='utf-8'), dtype="str", delimiter="\t")
print(len(df))
print(df.shape)
print(df[0])

23470
(23470, 3)
['<Burbank,_California>' 'rdf:type'
 '<wikicat_Populated_places_established_in_1911>']


In [40]:
count = 0
f = open("./yago10/types/types_test_004.nt","w+", encoding='utf-8')

for x in range(0, len(df)):
    s = df[x][0].replace("<","").replace(">","")
    p = df[x][1].replace("<","").replace(">","").replace("rdf:","")
    o = df[x][2].replace("<","").replace(">","")
    s = "<http://yago-knowledge.org/resource/" + s + ">"
    p = "<http://www.w3.org/1999/02/22-rdf-syntax-ns#" + p + ">"
    o = "<http://yago-knowledge.org/resource/" + o + ">"
    f.write("%s\t%s\t%s .\n" % (s,p,o))
    count = count + 1

f.close()
print(count)

23470
